In [61]:
import pandas as pd
import re
import numpy as np

### Loading Raw Data

In [62]:
def load_raw_data() -> dict:
    people = pd.read_csv("../data/raw/people.csv")
    salary = pd.read_csv("../data/raw/salary.csv")
    descriptions = pd.read_csv("../data/raw/descriptions.csv")
    return people, salary, descriptions

In [63]:
people, salary, descriptions = load_raw_data()

### Exploring Data

In [64]:
people.head()

,id,Age,Gender,Education Level,Job Title,Years of Experience
0,0,32.0,Male,Bachelor's,Software Engineer,5.0
1,1,28.0,Female,Master's,Data Analyst,3.0
2,2,45.0,Male,PhD,Senior Manager,15.0
3,3,36.0,Female,Bachelor's,Sales Associate,7.0
4,4,52.0,Male,Master's,Director,20.0


In [65]:
print(f"People\n\nShape: {people.shape}")
print(f"\nNull Values:\n{people.isnull().sum()}")
print(f"\nNmb of Unique Values:\n{people.nunique()}")

People

Shape: (375, 6)

Null Values:
id                     0
Age                    5
Gender                 5
Education Level        5
Job Title              5
Years of Experience    2
dtype: int64

Nmb of Unique Values:
id                     375
Age                     31
Gender                   2
Education Level          3
Job Title              173
Years of Experience     28
dtype: int64


In [66]:
salary.head()

,id,Salary
0,0,90000.0
1,1,65000.0
2,2,150000.0
3,3,60000.0
4,4,200000.0


In [67]:
print(f"Salary\n\nShape: {salary.shape}")
print(f"\nNull Values:\n{salary.isnull().sum()}")
print(f"\nNmb of Unique Values:\n{salary.nunique()}")

Salary

Shape: (375, 2)

Null Values:
id        0
Salary    2
dtype: int64

Nmb of Unique Values:
id        375
Salary     36
dtype: int64


In [68]:
descriptions.head()

,id,Description
0,0,I am a 32-year-old male working as a Software ...
1,1,I am a 28-year-old data analyst with a Master'...
2,2,I am a 45-year-old Senior Manager with a PhD a...
3,3,I am a 36-year-old female Sales Associate with...
4,4,I am a 52-year-old male with over two decades ...


In [69]:
print(f"Descriptions\n\nShape: {descriptions.shape}")
print(f"\nNull Values:\n{descriptions.isnull().sum()}")
print(f"\nNmb of Unique Values:\n{descriptions.nunique()}")

Descriptions

Shape: (375, 2)

Null Values:
id             0
Description    3
dtype: int64

Nmb of Unique Values:
id             375
Description    372
dtype: int64


### Merging Tables

In [70]:
df_raw = (
    people
    .merge(salary, on="id", how="left")
    .merge(descriptions, on="id", how="left")
)
df_raw.head()

,id,Age,Gender,Education Level,Job Title,Years of Experience,Salary,Description
0,0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0,I am a 32-year-old male working as a Software ...
1,1,28.0,Female,Master's,Data Analyst,3.0,65000.0,I am a 28-year-old data analyst with a Master'...
2,2,45.0,Male,PhD,Senior Manager,15.0,150000.0,I am a 45-year-old Senior Manager with a PhD a...
3,3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0,I am a 36-year-old female Sales Associate with...
4,4,52.0,Male,Master's,Director,20.0,200000.0,I am a 52-year-old male with over two decades ...


In [71]:
df_raw_cols = (
    df_raw
    .rename(
        columns={
            "Age": "age",
            "Gender": "gender",
            "Education Level": "education_level",
            "Job Title": "job_title",
            "Years of Experience": "years_of_experience",
            "Salary": "salary",
            "Description": "description"
        }
    )
).copy()
df_raw_cols.head()

,id,age,gender,education_level,job_title,years_of_experience,salary,description
0,0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0,I am a 32-year-old male working as a Software ...
1,1,28.0,Female,Master's,Data Analyst,3.0,65000.0,I am a 28-year-old data analyst with a Master'...
2,2,45.0,Male,PhD,Senior Manager,15.0,150000.0,I am a 45-year-old Senior Manager with a PhD a...
3,3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0,I am a 36-year-old female Sales Associate with...
4,4,52.0,Male,Master's,Director,20.0,200000.0,I am a 52-year-old male with over two decades ...


In [72]:
df_raw_cols.dtypes

id                       int64
age                    float64
gender                  object
education_level         object
job_title               object
years_of_experience    float64
salary                 float64
description             object
dtype: object

In [73]:
df = df_raw_cols.copy()
# df["age"] = df_raw_cols["age"].astype(int)
# df["years_of_experience"] = df_raw_cols["years_of_experience"].astype(int)
# df["salary"] = df_raw_cols["salary"].astype(int)
# df.head()

# NO SE PUEDEN TRANSFORMAR DE TIPO LAS COLUMNAS CON MISSING VALUES

### Valores Duplicados

In [74]:
print(df.shape)
print(df.drop_duplicates().shape)
# No hay duplicados

(375, 8)
(375, 8)


### Manejo de Valores Nulos

In [75]:
df.isnull().sum()

id                     0
age                    5
gender                 5
education_level        5
job_title              5
years_of_experience    2
salary                 2
description            3
dtype: int64

Se intentará encontrar la información faltante utilizando la descripcion

In [76]:
def extract_capital_words(s: str) -> str:
    return " ".join(re.findall(r'\b[A-Z][a-zA-Z]*\b', s))

# r'...' → raw string. Python lo interpreta tal cual, sin escapar \.
# \b → límite de palabra (marca el inicio o fin de una palabra).
# [A-Z] → una sola letra mayúscula.
# [a-zA-Z]* → cero o más letras (mayúsculas o minúsculas).
# \b → fin de palabra.

In [77]:
# No se puede aplicar regex a columna con valores nulos
df["description"] = df["description"].fillna("no-description")

# Eliminar primera palabra si empieza con mayúscula
df["clean_description"] = (
    df["description"]
    .str.replace(r'^[A-Z][a-zA-Z]*\b\s*', '', regex=True)
)

# Eliminar palabras después de un punto que empiezan con mayúscula
df["clean_description"] = (
    df["clean_description"]
    .str.replace(r'\.\s*\b[A-Z][a-zA-Z]*\b', '', regex=True))

# Extrar palabras con mayuscula
df["keywords"] = df["clean_description"].apply(extract_capital_words)

df["keywords"] = df["keywords"].replace(r'\b(My|I)\b', '', regex=True)
df.head()

,id,age,gender,education_level,job_title,years_of_experience,salary,description,clean_description,keywords
0,0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0,I am a 32-year-old male working as a Software ...,am a 32-year-old male working as a Software En...,Software Engineer Bachelor Computer Science P...
1,1,28.0,Female,Master's,Data Analyst,3.0,65000.0,I am a 28-year-old data analyst with a Master'...,am a 28-year-old data analyst with a Master's ...,Master
2,2,45.0,Male,PhD,Senior Manager,15.0,150000.0,I am a 45-year-old Senior Manager with a PhD a...,am a 45-year-old Senior Manager with a PhD and...,Senior Manager PhD
3,3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0,I am a 36-year-old female Sales Associate with...,am a 36-year-old female Sales Associate with a...,Sales Associate Bachelor
4,4,52.0,Male,Master's,Director,20.0,200000.0,I am a 52-year-old male with over two decades ...,am a 52-year-old male with over two decades of...,Director Master


In [78]:
df["age_new"] = df["age"]
df["age_new"] = (
    df["age_new"]
    .fillna(
        df["clean_description"]
        .str.extract(r'(\d+)(?:-?year-old| years old)')[0]
        .astype(float)
    )
)
df.isnull().sum()

id                     0
age                    5
gender                 5
education_level        5
job_title              5
years_of_experience    2
salary                 2
description            0
clean_description      0
keywords               0
age_new                3
dtype: int64

In [79]:
# Crear un patrón regex a partir de la lista
pattern = r"(Bachelor's|Master's|PhD)"

# Buscar y asignar el valor encontrado
df["education_level_new"] = df["education_level"]
df["education_level_new"] = (
    df["education_level_new"]
    .fillna(
        df["clean_description"]
        .str.extract(pattern, expand=False)
    )
)
df.isnull().sum()

id                     0
age                    5
gender                 5
education_level        5
job_title              5
years_of_experience    2
salary                 2
description            0
clean_description      0
keywords               0
age_new                3
education_level_new    2
dtype: int64

In [80]:
levels = r"(Bachelor|Master|PhD)"

def split_job_edu(s):
    match = re.search(levels, s)
    if not match:
        return s  # sin educación, todo es job_title
    
    edu = match.group(1)
    start, end = match.span()
    
    # Si el nivel educativo está al inicio
    if start == 0:
        job = s[end:].strip()
    else:
        job = s[:start].strip()
    
    return job

df["job_title_new"] = df["job_title"]
df["job_title_new"] = (
    df["job_title_new"]
    .fillna(
        df["keywords"]
        .apply(lambda x: split_job_edu(x))
    )
)

df.isna().sum()

id                     0
age                    5
gender                 5
education_level        5
job_title              5
years_of_experience    2
salary                 2
description            0
clean_description      0
keywords               0
age_new                3
education_level_new    2
job_title_new          0
dtype: int64

In [81]:
df[df.isna().any(axis=1)]

,id,age,gender,education_level,job_title,years_of_experience,salary,description,clean_description,keywords,age_new,education_level_new,job_title_new
51,51,33.0,Male,Master's,NaN,7.0,85000.0,I am a 33-year-old Business Intelligence Analy...,am a 33-year-old Business Intelligence Analyst...,Business Intelligence Analyst Master BI SQL Ta...,33.0,Master's,Business Intelligence Analyst
60,60,51.0,Female,Master's,NaN,23.0,170000.0,I am a 51-year-old female with a Master's degr...,am a 51-year-old female with a Master's degree...,Master Director Operations,51.0,Master's,Director Operations
139,139,43.0,Female,NaN,Senior Product Marketing Manager,14.0,120000.0,I am a 43-year-old Senior Product Marketing Ma...,am a 43-year-old Senior Product Marketing Mana...,Senior Product Marketing Manager Master,43.0,Master's,Senior Product Marketing Manager
172,172,NaN,NaN,NaN,NaN,NaN,NaN,"As an employee, I bring a wealth of diverse ex...","an employee, I bring a wealth of diverse exper...",,NaN,NaN,
219,219,40.0,NaN,Bachelor's,Senior Sales Representative,12.0,100000.0,I am a 40-year-old Senior Sales Representative...,am a 40-year-old Senior Sales Representative w...,Senior Sales Representative Bachelor,40.0,Bachelor's,Senior Sales Representative
221,221,NaN,Female,Bachelor's,Junior Social Media Specialist,3.0,45000.0,I am a 31-year-old female currently working as...,am a 31-year-old female currently working as a...,Junior Social Media Specialist Bachelor,31.0,Bachelor's,Junior Social Media Specialist
225,225,40.0,NaN,Bachelor's,Senior Marketing Manager,11.0,105000.0,I am a 40-year-old Senior Marketing Manager wi...,am a 40-year-old Senior Marketing Manager with...,Senior Marketing Manager Bachelor,40.0,Bachelor's,Senior Marketing Manager
235,235,32.0,NaN,Bachelor's,Junior Sales Representative,3.0,45000.0,As a 32-year-old Junior Sales Representative w...,a 32-year-old Junior Sales Representative with...,Junior Sales Representative Bachelor,32.0,Bachelor's,Junior Sales Representative
260,260,NaN,NaN,NaN,NaN,NaN,NaN,"As an employee, I bring a unique blend of skil...","an employee, I bring a unique blend of skills ...",,NaN,NaN,
261,261,37.0,Female,NaN,Senior Financial Manager,10.0,120000.0,I am a 37-year-old Senior Financial Manager wi...,am a 37-year-old Senior Financial Manager with...,Senior Financial Manager Bachelor,37.0,Bachelor's,Senior Financial Manager


In [82]:
df = df.drop([172,260])
df.isnull().sum()

id                     0
age                    3
gender                 3
education_level        3
job_title              3
years_of_experience    0
salary                 0
description            0
clean_description      0
keywords               0
age_new                1
education_level_new    0
job_title_new          0
dtype: int64

In [83]:
df_new = df[
    [
        "id",
        "age_new",
        "gender",
        "education_level_new",
        "job_title_new",
        "years_of_experience",
        "salary",
        "keywords",
        "clean_description"
    ]
]
df_new.head()

,id,age_new,gender,education_level_new,job_title_new,years_of_experience,salary,keywords,clean_description
0,0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0,Software Engineer Bachelor Computer Science P...,am a 32-year-old male working as a Software En...
1,1,28.0,Female,Master's,Data Analyst,3.0,65000.0,Master,am a 28-year-old data analyst with a Master's ...
2,2,45.0,Male,PhD,Senior Manager,15.0,150000.0,Senior Manager PhD,am a 45-year-old Senior Manager with a PhD and...
3,3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0,Sales Associate Bachelor,am a 36-year-old female Sales Associate with a...
4,4,52.0,Male,Master's,Director,20.0,200000.0,Director Master,am a 52-year-old male with over two decades of...


### Working With New Dataframe

In [87]:
df_new = df_new.copy()  # si es un slice, asegurate de usar .copy()
df_new["level"] = pd.Series(dtype="object")  # columna vacía tipo string

df_new.loc[df_new["job_title_new"].str.contains(r"\bJunior\b", case=False, na=False), "level"] = "Junior"
df_new.loc[df_new["job_title_new"].str.contains(r"\bSenior\b", case=False, na=False), "level"] = "Senior"

df_new.loc[df_new["keywords"].str.contains(r"\bJunior\b", case=False, na=False), "level_keyword"] = "Junior"
df_new.loc[df_new["keywords"].str.contains(r"\bSenior\b", case=False, na=False), "level_keyword"] = "Senior"

df_new["level"] = (
    df_new["level"]
    .fillna(df_new["level_keyword"])
    .fillna("Not-Applicable")
)
df_new = (
    df_new
    .drop(columns=["level_keyword"])
)
df_new.isnull().sum()

id                     0
age_new                1
gender                 3
education_level_new    0
job_title_new          0
years_of_experience    0
salary                 0
keywords               0
clean_description      0
level                  0
dtype: int64

In [88]:
df_new.head()

,id,age_new,gender,education_level_new,job_title_new,years_of_experience,salary,keywords,clean_description,level
0,0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0,Software Engineer Bachelor Computer Science P...,am a 32-year-old male working as a Software En...,Not-Applicable
1,1,28.0,Female,Master's,Data Analyst,3.0,65000.0,Master,am a 28-year-old data analyst with a Master's ...,Not-Applicable
2,2,45.0,Male,PhD,Senior Manager,15.0,150000.0,Senior Manager PhD,am a 45-year-old Senior Manager with a PhD and...,Senior
3,3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0,Sales Associate Bachelor,am a 36-year-old female Sales Associate with a...,Not-Applicable
4,4,52.0,Male,Master's,Director,20.0,200000.0,Director Master,am a 52-year-old male with over two decades of...,Not-Applicable


In [99]:
df_new["job_title_clean"] = (
    df_new["job_title_new"]
    .str.replace(r"\b(Senior|Junior)\b", "", regex=True)
    .str.strip()
)
df_new.head()

,id,age_new,gender,education_level_new,job_title_new,years_of_experience,salary,keywords,clean_description,level,job_title_clean
0,0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0,Software Engineer Bachelor Computer Science P...,am a 32-year-old male working as a Software En...,Not-Applicable,Software Engineer
1,1,28.0,Female,Master's,Data Analyst,3.0,65000.0,Master,am a 28-year-old data analyst with a Master's ...,Not-Applicable,Data Analyst
2,2,45.0,Male,PhD,Senior Manager,15.0,150000.0,Senior Manager PhD,am a 45-year-old Senior Manager with a PhD and...,Senior,Manager
3,3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0,Sales Associate Bachelor,am a 36-year-old female Sales Associate with a...,Not-Applicable,Sales Associate
4,4,52.0,Male,Master's,Director,20.0,200000.0,Director Master,am a 52-year-old male with over two decades of...,Not-Applicable,Director


In [108]:
from collections import Counter
all_words = " ".join(df_new["job_title_clean"]).split()

# Contar la frecuencia de cada palabra
word_counts = Counter(all_words)

# Convertir a DataFrame para ver mejor
word_counts_df = pd.DataFrame(word_counts.items(), columns=["Word", "Count"]).sort_values(by="Count", ascending=False)
# word_counts_df.head(50)

In [113]:
df_iter = df_new.copy()  # trabajar sobre una copia
top_words = []
words = []
for i in range(8):
    # 1️⃣ Unir todas las filas y separar en palabras
    all_words = " ".join(df_iter["job_title_clean"].dropna()).split()
    
    if not all_words:  # Si ya no hay palabras, salimos
        break
    
    # 2️⃣ Contar frecuencia
    word_counts = Counter(all_words)
    
    # 3️⃣ Palabra más común
    word, count = word_counts.most_common(1)[0]
    
    # 4️⃣ Guardar en la lista
    top_words.append((word, count))
    words.append(word)
    
    # 5️⃣ Dropear todas las filas que contengan esa palabra
    df_iter = df_iter[~df_iter["job_title_clean"].str.contains(rf"\b{word}\b", case=False, na=False)]

# Mostrar resultado
print(top_words)

[('Manager', 96), ('Analyst', 69), ('Director', 42), ('Coordinator', 28), ('Specialist', 20), ('Engineer', 17), ('Scientist', 16), ('Designer', 15)]


In [122]:
top_related_words_iterative = {}
word_nmb = 0
all_words = []
for word in words:
    # 1️⃣ Filtrar el dataframe por filas que contengan la palabra actual
    df_filtered = df_new[df_new["job_title_clean"].str.contains(rf"\b{word}\b", case=False, na=False)].copy()
    
    top3 = []
    word_nmb += 1
    if word_nmb < 3:
        n = 4
    else:
        n = 3
    for _ in range(n):
        # Si ya no quedan palabras, salir
        if df_filtered.empty:
            break
        
        # 2️⃣ Contar todas las palabras en el subset
        all_words = " ".join(df_filtered["job_title_clean"].dropna()).split()
        if not all_words:
            break
        
        word_counts = Counter(all_words)
        
        # 3️⃣ Eliminar la palabra original usada para filtrar si está presente
        word_counts.pop(word, None)

        if not word_counts:
            break
        
        # 4️⃣ Seleccionar la palabra más frecuente
        top_word, count = word_counts.most_common(1)[0]
        if top_word != "of":
            top3.append((top_word, count))
        else:
            top_word, count = word_counts.most_common(2)[1]
            top3.append((top_word, count))

        # 5️⃣ Dropear filas que contengan esa palabra para la siguiente iteración
        df_filtered = df_filtered[~df_filtered["job_title_clean"].str.contains(rf"\b{top_word}\b", case=False, na=False)]
    
    top_related_words_iterative[word] = top3

# Mostrar resultados
for w, related in top_related_words_iterative.items():
    print(f"{w}: {related}")

Manager: [('Marketing', 17), ('Project', 16), ('Product', 13), ('Operations', 11)]
Analyst: [('Business', 23), ('Financial', 15), ('Marketing', 14), ('Operations', 8)]
Director: [('Marketing', 14), ('Operations', 12), ('Human', 4)]
Coordinator: [('Marketing', 12), ('Project', 5), ('Operations', 5)]
Specialist: [('Marketing', 10), ('Support', 4), ('Social', 2)]
Engineer: [('Software', 8), ('Data', 4), ('Project', 1)]
Scientist: [('Data', 9), ('Research', 3), ('Principal', 1)]
Designer: [('Product', 6), ('UX', 5), ('Graphic', 2)]


In [137]:
# Lista para guardar resultados
result_keys = []
result_words =[]
for key, tuples in top_related_words_iterative.items():
    result_keys.append(key)
    for t in tuples:
        first_word = t[0]  # primer elemento de cada tupla
        result_words.append(first_word)

In [141]:
main_words = list(set(result_keys))
main_words

['Manager',
 'Coordinator',
 'Scientist',
 'Engineer',
 'Specialist',
 'Director',
 'Analyst',
 'Designer']

In [142]:
secondary_words = list(set(result_words))
secondary_words

['Software',
 'Product',
 'Social',
 'Operations',
 'UX',
 'Human',
 'Graphic',
 'Business',
 'Marketing',
 'Data',
 'Project',
 'Research',
 'Financial',
 'Support',
 'Principal']

In [144]:
# Crear patrón regex: \bWORD\b para que busque palabras completas
pattern_main = r'\b(' + '|'.join(main_words) + r')\b'
pattern_secondary = r'\b(' + '|'.join(secondary_words) + r')\b'

# Crear la nueva columna
df_new['main_word'] = df_new['job_title_clean'].str.extract(pattern_main)
df_new['secondary_word'] = df_new['job_title_clean'].str.extract(pattern_secondary)

df_new.head()

,id,age_new,gender,education_level_new,job_title_new,years_of_experience,salary,keywords,clean_description,level,job_title_clean,main_word,secondary_word
0,0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0,Software Engineer Bachelor Computer Science P...,am a 32-year-old male working as a Software En...,Not-Applicable,Software Engineer,Engineer,Software
1,1,28.0,Female,Master's,Data Analyst,3.0,65000.0,Master,am a 28-year-old data analyst with a Master's ...,Not-Applicable,Data Analyst,Analyst,Data
2,2,45.0,Male,PhD,Senior Manager,15.0,150000.0,Senior Manager PhD,am a 45-year-old Senior Manager with a PhD and...,Senior,Manager,Manager,NaN
3,3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0,Sales Associate Bachelor,am a 36-year-old female Sales Associate with a...,Not-Applicable,Sales Associate,NaN,NaN
4,4,52.0,Male,Master's,Director,20.0,200000.0,Director Master,am a 52-year-old male with over two decades of...,Not-Applicable,Director,Director,NaN


In [145]:
df_new.isnull().sum()

id                      0
age_new                 1
gender                  3
education_level_new     0
job_title_new           0
years_of_experience     0
salary                  0
keywords                0
clean_description       0
level                   0
job_title_clean         0
main_word              70
secondary_word         97
dtype: int64